In [53]:
import random
import torch
from tokenization_jp_mod import BertJapaneseTokenizer
from transformers.modeling_bert import BertForMaskedLM

In [3]:
text = '慢性膵炎では，糖尿病性ケトアシドーシス(DKA)の合併は比較的稀である．'

# Model path
PathToBertJpMecabIpadic = '/PATH/TO/BERT/MECAB/IPADIC/DIR/'
PathToBertJpMecabIpadicNeologdJmedic = '/PATH/TO/BERT/MECAB/IPADIC/NEOLOGD/JMEDIC/DIR/'
PathToBertJpSentencepiece = '/PATH/TO/BERT/MECAB/SENTENCEPIECE/DIR/'

# MeCab dictionary
PathToNeologd = '/path/to/mecab/dic/mecab-ipadic-neologd/'
PathToJmedic = '/path/to/mecab/dic/manbyo/MANBYO_201907_Dic-utf8.dic'

In [288]:
def prediction_masked_sentence(sentence, tokenizer, model):
    mask_token_id = tokenizer.mask_token_id
    
    input_ids = tokenizer.encode(text, return_tensors='pt')
    print(' '.join(tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])))
    masked_index = random.randint(1, len(input_ids[0]) - 2)
    input_ids[0][masked_index] = mask_token_id
    print(' '.join(tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])))
    
    result = model(input_ids)
    topk = torch.nn.Softmax(dim=1)(result[0][:, masked_index]).topk(5)
    scores = topk.values.tolist()[0]
    pred_ids = topk.indices.tolist()[0]
    for i, pred_id in enumerate(pred_ids):
        output_ids = input_ids.tolist()[0]
        output_ids[masked_index] = pred_id
        
        pred_sentence = tokenizer.convert_ids_to_tokens(output_ids)
        pred_sentence = [token if n != masked_index else '【' + token + '】'
                         for n, token in enumerate(pred_sentence)]
        print('{:.1f} %'.format(scores[i] * 100).rjust(7), ' '.join(pred_sentence))
        
    return result

# Publicly available Japanese BERT model

In [291]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking',
                                                  mecab_kwargs={
                                                     "mecab_dic": 'ipadic',
                                                 })
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [416]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
[CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス [MASK] D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 98.8 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス 【(】 D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.7 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス 【(,】 D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.1 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス 【(-】 D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.0 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス 【(=】 D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.0 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス 【,】 D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]


# medBERTjp - MeCab with IPAdic

In [262]:
model_path = PathToBertJpMecabIpadic
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path, 
                                                 mecab_kwargs={
                                                     "mecab_dic": 'ipadic',
                                                 })
model = BertForMaskedLM.from_pretrained(model_path)

In [263]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] 慢性 膵炎 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
[CLS] 慢性 [MASK] で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 25.5 % [CLS] 慢性 【型】 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 15.4 % [CLS] 慢性 【膵炎】 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 12.6 % [CLS] 慢性 【中毒】 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 10.5 % [CLS] 慢性 【例】 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  4.3 % [CLS] 慢性 【疾患】 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]


# medBERTjp - MeCab with IPAdic-NEologd + J-MeDic

In [240]:
model_path = PathToBertJpMecabIpadicNeologdJmedic
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path,
                                                  word_tokenizer_type='mecab',
                                                  mecab_kwargs={
                                                      'mecab_dic_path': PathToNeologd,
                                                      'mecab_userdic_path': PathToJmedic,
                                                  }
                                                 )
model = BertForMaskedLM.from_pretrained(model_path)

In [261]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] 慢性膵炎 では , 糖尿病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
[CLS] [MASK] では , 糖尿病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 45.5 % [CLS] 【糖尿病患者】 では , 糖尿病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 39.5 % [CLS] 【糖尿病】 では , 糖尿病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  1.7 % [CLS] 【アルコール依存症】 では , 糖尿病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  1.4 % [CLS] 【1型糖尿病】 では , 糖尿病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.8 % [CLS] 【ケトアシドーシス】 では , 糖尿病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]


# medBERTjp - SentencePiece

In [264]:
model_path = PathToBertJpSentencepiece
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path,
                                                  word_tokenizer_type='sp',
                                                 )
model = BertForMaskedLM.from_pretrained(model_path)

/data1/wada/workspace/model_uploaded/medBERTjp_L12_H768_A12_sentencepiece/vocab.model
Loaded a trained SentencePiece model.


In [287]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
[CLS] ▁ [MASK] では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
 22.4 % [CLS] ▁ 【若年者】 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
  5.5 % [CLS] ▁ 【それ以外】 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
  4.3 % [CLS] ▁ 【糖尿病患者】 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
  3.9 % [CLS] ▁ 【入院患者】 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
  3.5 % [CLS] ▁ 【救急外来】 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
